## PostMates Data Exercise - Anirban Bhattacharyya

## Background

Jumpman23 is an on-demand delivery platform connecting “Jumpmen” and customers purchasing a variety of goods. Jumpman23 will send Jumpmen to merchants to purchase and pickup any items requested by the customer. Whenever possible, Jumpman23 will order the requested items ahead to save the Jumpmen time. Each time a Jumpman23 delivery is
completed, a record is saved to the Jumpman23 database that contains information about that delivery. Jumpman23 is growing fast and has just launched in its newest market -- New York City.

## Challenge

The CEO of Jumpman23 has just asked you “how are things going in New York”. He has mentioned to you that he’s heard reports of data integrity issues. Please think through the CEO’s ask and present to both the CEO and the CTO, in any format you choose, an analysis of the market. In addition, dive into the reports on data integrity issues and if they indeed exist, outline where they may be and how they may impact the analysis. The CEO is a visual learner, the CTO loves to see code and technical work, and both are obsessed with maps.

## Available data

You have been provided one small csv file with a sample of data. This file includes:

● Job_ID -- > a unique identifier of a delivery

● Customer_id → a unique identifier for the Jumpman23 customer

● Jumpman_id → a unique identifier for the Jumpman who completed the delivery

● vehicle_type → The method of transport the Jumpman used to complete the delivery

● pickup_place → The name of the Pickup location

● place_category → A categorization of the Pickup location

● Item_name → the name of the item requested

● Item_quantity → how many of that item was requested

● Item_category_name → categorization provided by merchant, think “appetizers”,“soups” etc

● How_long_it_took_to_order → how long it took to place the order [interval]

● pickup_lat → the coordinates of the pickup location

● pickup_lon → the coordinates of the pickup location

● dropoff_lat → the coordinations of the dropoff location

● dropoff_lon → the coordinations of the dropoff location

● when_the_delivery_started→ localized timestamp representing when the delivery began

● when_the_Jumpman_arrived_at_pickup → localized timestamp representing when the Jumpman arrived at the pickup location

● when_the_Jumpman_left_pickup → localized timestamp representing when the Jumpman left the pickup location

● when_the_Jumpman_arrived_at_dropoff → localized timestamp representing when the Jumpman reached the customer

*There are 18 columns x 5983 rows of data*.

We should be able to see the geographic story of what this data is trying to tell us, as well as gain more insight into how orders are taking place. Finally we will go deeper to try and detect where the data integrity issues are taking place.

In [1]:
# Importing libraries for data analysis
import pandas as pd
import numpy as np

# Reading dataset into a pandas dataframe
data = pd.read_csv('analyze_me.csv')

In [2]:
# Import libraries needed for data visualization
import folium
from folium import plugins
import plotly.offline as py 
import plotly.graph_objs as go
import cufflinks as cf
import plotly.figure_factory as ff
cf.go_offline()
py.init_notebook_mode(connected=True)

## Exploratory Analysis

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5983 entries, 0 to 5982
Data columns (total 18 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   delivery_id                          5983 non-null   int64  
 1   customer_id                          5983 non-null   int64  
 2   jumpman_id                           5983 non-null   int64  
 3   vehicle_type                         5983 non-null   object 
 4   pickup_place                         5983 non-null   object 
 5   place_category                       5100 non-null   object 
 6   item_name                            4753 non-null   object 
 7   item_quantity                        4753 non-null   float64
 8   item_category_name                   4753 non-null   object 
 9   how_long_it_took_to_order            3038 non-null   object 
 10  pickup_lat                           5983 non-null   float64
 11  pickup_lon                    

In [4]:
data.head()

,delivery_id,customer_id,jumpman_id,vehicle_type,pickup_place,place_category,item_name,item_quantity,item_category_name,how_long_it_took_to_order,pickup_lat,pickup_lon,dropoff_lat,dropoff_lon,when_the_delivery_started,when_the_Jumpman_arrived_at_pickup,when_the_Jumpman_left_pickup,when_the_Jumpman_arrived_at_dropoff
0,1457973,327168,162381,van,Melt Shop,American,Lemonade,1.0,Beverages,00:19:58.582052,40.744607,-73.990742,40.752073,-73.985370,2014-10-26 13:51:59.898924,NaN,NaN,2014-10-26 14:52:06.313088
1,1377056,64452,104533,bicycle,Prince Street Pizza,Pizza,Neapolitan Rice Balls,3.0,Munchables,00:25:09.107093,40.723080,-73.994615,40.719722,-73.991858,2014-10-16 21:58:58.65491,2014-10-16 22:26:02.120931,2014-10-16 22:48:23.091253,2014-10-16 22:59:22.948873
2,1476547,83095,132725,bicycle,Bareburger,Burger,Bare Sodas,1.0,Drinks,00:06:44.541717,40.728478,-73.998392,40.728606,-73.995143,2014-10-28 21:39:52.654394,2014-10-28 21:37:18.793405,2014-10-28 21:59:09.98481,2014-10-28 22:04:40.634962
3,1485494,271149,157175,bicycle,Juice Press,Juice Bar,OMG! My Favorite Juice!,1.0,Cold Pressed Juices,NaN,40.738868,-74.002747,40.751257,-74.005634,2014-10-30 10:54:11.531894,2014-10-30 11:04:17.759577,2014-10-30 11:16:37.895816,2014-10-30 11:32:38.090061
4,1327707,122609,118095,bicycle,Blue Ribbon Sushi,Japanese,Spicy Tuna & Tempura Flakes,2.0,Maki (Special Rolls),00:03:45.035418,40.726110,-74.002492,40.709323,-74.015867,2014-10-10 00:07:18.450505,2014-10-10 00:14:42.702223,2014-10-10 00:25:19.400294,2014-10-10 00:48:27.150595


### Times of the day with most demand

In [5]:
data.when_the_delivery_started= pd.to_datetime(data.when_the_delivery_started)
data.when_the_Jumpman_arrived_at_pickup = pd.to_datetime(data.when_the_Jumpman_arrived_at_pickup)
data.when_the_Jumpman_left_pickup = pd.to_datetime(data.when_the_Jumpman_left_pickup)
data.when_the_Jumpman_arrived_at_dropoff= pd.to_datetime(data.when_the_Jumpman_arrived_at_dropoff)

In [6]:
# The goal is to look at the distribution of orders by the hour of the day when the order is placed
def getHours(row):
    return row.hour
data.when_the_delivery_started.apply(getHours).value_counts(normalize=1)

19    0.157112
18    0.125522
20    0.119505
17    0.076383
21    0.075882
12    0.068695
11    0.053986
13    0.053151
16    0.048304
14    0.044292
15    0.042286
22    0.035768
10    0.027578
23    0.023232
9     0.014708
0     0.012201
1     0.006686
8     0.005181
2     0.003677
7     0.003343
3     0.001170
4     0.001003
6     0.000334
Name: when_the_delivery_started, dtype: float64

In [7]:
data.when_the_delivery_started.apply(getHours).value_counts().iplot(kind='bar', title='Hour by Hour Visualization of Demand')

**Peak hours are from 5 pm - 9 pm and accounts for 54% of all orders**

### Average time to order

In [8]:
data['how_long_it_took_to_order'] = pd.to_datetime(data.how_long_it_took_to_order, format = '%H:%M:%S.%f')

In [9]:
def get_min(row):
    return row.minute*60.0 + row.second
minute = data.how_long_it_took_to_order.apply(get_min)
data['min_to_order'] = round(minute/60,2)

In [10]:
data.min_to_order.describe()

count    3038.000000
mean        7.751754
std         5.313122
min         1.370000
25%         4.430000
50%         6.280000
75%         9.170000
max        58.030000
Name: min_to_order, dtype: float64

In [11]:
data.min_to_order.iplot(kind='box', title= 'Range of order times', yTitle = 'time in minutes')

While a majority of orders are completed within 10 minutes, the data shows that there is a significant amount of variance and upper fence outliers that are most likely impeding the experience of the product. This could be a result of a user experience issue:

1. Technical bug or slowdown of the consumer facing application
2. Story-Flow or Layout issue causing choice paralysis

Make the order experience easier and more intuitive. Reducing time-spent to order is a very important metric to reduce churn and increase retention(user stickiness) & revival(previous user returns).

Building a recommender system should introduce personalization and cut down on decision time as the product learns the user’s preferences. This would make the product much more easier to use, as well as feeling the product is very novel and innovative.

In [20]:
from datetime import timedelta
data['t_start_pickup_m'] = (data.when_the_Jumpman_arrived_at_pickup - data.when_the_delivery_started)/timedelta(minutes=1)
data['t_pickup_duration_m'] = (data.when_the_Jumpman_left_pickup - data.when_the_Jumpman_arrived_at_pickup)/timedelta(minutes=1)
data['t_dropoff_duration_m'] = (data.when_the_Jumpman_arrived_at_dropoff - data.when_the_Jumpman_left_pickup)/timedelta(minutes=1)
data[['t_start_pickup_m','t_pickup_duration_m','t_dropoff_duration_m']].iplot(kind='box')

Time Spent Picking up is the **least consistent** of the duration ranges. This is to be expected as Postmate’s does not have significant jurisdiction in influencing the operations of restaurant food production. However changing to a stocking model for the most popular items, may introduce efficiency towards this bottleneck.

#### Most popular places based on moving the most inventory

In [21]:
places = data.groupby('pickup_place').sum()
places.sort_values('item_quantity', ascending = 0).item_quantity[:20].iplot(kind='bar',title='Popular Restaurants')

Supply and demand logistics are incredibly important when running a business like Postmates. Being able to accurately predict and accommodate when supply and demand rise and fall are instrumental to providing a magical experience for the end user.

One simple strategy would be to look in the data for the hottest restaurants and buy the most frequently ordered items ahead of time during peak hours to alleviate order congestion and reduce upper fence variability by relying on the restaurant partners.

#### Quantity per order

In [22]:
data.item_quantity.value_counts().sort_index()

1.0     3980
2.0      570
3.0      112
4.0       54
5.0       13
6.0       14
7.0        1
8.0        4
12.0       1
15.0       3
16.0       1
Name: item_quantity, dtype: int64

In [17]:
data.item_quantity.value_counts(normalize=1).sort_index()

1.0     0.837366
2.0     0.119924
3.0     0.023564
4.0     0.011361
5.0     0.002735
6.0     0.002946
7.0     0.000210
8.0     0.000842
12.0    0.000210
15.0    0.000631
16.0    0.000210
Name: item_quantity, dtype: float64

**Over 83% of customers prefer to order 1 item**

In [23]:
pop_categories = data.groupby('place_category').sum()
pop_categories['item_quantity'].sort_values(ascending=False)[:15].iplot(kind='bar', title='Popular Categories')

Burgers, Japanese and Italian are the top 3 most popular categories in terms of orders placed.

In [25]:
veh = data.vehicle_type.value_counts(normalize=0)
veh

bicycle       4274
car           1215
walker         274
van             76
scooter         75
truck           48
motorcycle      21
Name: vehicle_type, dtype: int64

In [26]:
pieData = [go.Pie(labels = veh.index, values = veh )]
py.iplot(go.Figure(data=pieData, layout = go.Layout(title= 'Most popular modes of transport for Jumpman')))

Bicycle’s are overwhelmingly the preferred mode of Transport for Jumpman in NYC. The transport decision factors in economic feasibility and the right tools to meet expectations of the job given the city road layout and current municipalities of the city.

## Visual heatmap of activity

In [27]:
m = folium.Map([40.744607, -73.990742], zoom_start=12)
n = folium.Map([40.744607, -73.990742], zoom_start=12)

#### Pickup Heatzones

In [28]:
stationArr = data[['pickup_lat', 'pickup_lon']]

# plot heatmap
m.add_child(plugins.HeatMap(stationArr, radius=15))
m

#### Dropoff heatzones

In [30]:
stationArr = data[['dropoff_lat', 'dropoff_lon']]

# plot heatmap
n.add_child(plugins.HeatMap(stationArr, radius=15))
n

Based off the two graphics above, it is apparent that the dropoff zones are much more spread out(Queens, Brooklyn, Upper-Manhattan) than the pickup zones. The graphics are irrespective of time, and focus on aggregated interest points.

### Detecting Data Integrity issues

The CEO has made it known that he thinks there are some Data Integrity issues within the current technical system, and is requesting those issues be found. As we’ve seen in the Duration from Order to Delivery graphic from earlier, there are erroneous entries of data (otherwise we shouldn’t be getting negative durations).

#### How much data is missing in each column?

In [31]:
data.isnull().sum()

delivery_id                               0
customer_id                               0
jumpman_id                                0
vehicle_type                              0
pickup_place                              0
place_category                          883
item_name                              1230
item_quantity                          1230
item_category_name                     1230
how_long_it_took_to_order              2945
pickup_lat                                0
pickup_lon                                0
dropoff_lat                               0
dropoff_lon                               0
when_the_delivery_started                 0
when_the_Jumpman_arrived_at_pickup      550
when_the_Jumpman_left_pickup            550
when_the_Jumpman_arrived_at_dropoff       0
min_to_order                           2945
t_start_pickup_m                        550
t_pickup_duration_m                     550
t_dropoff_duration_m                    550
dtype: int64

In [32]:
data[data.isnull().any(axis=1)]

,delivery_id,customer_id,jumpman_id,vehicle_type,pickup_place,place_category,item_name,item_quantity,item_category_name,how_long_it_took_to_order,...,dropoff_lat,dropoff_lon,when_the_delivery_started,when_the_Jumpman_arrived_at_pickup,when_the_Jumpman_left_pickup,when_the_Jumpman_arrived_at_dropoff,min_to_order,t_start_pickup_m,t_pickup_duration_m,t_dropoff_duration_m
0,1457973,327168,162381,van,Melt Shop,American,Lemonade,1.0,Beverages,1900-01-01 00:19:58.582052,...,40.752073,-73.985370,2014-10-26 13:51:59.898924,NaT,NaT,2014-10-26 14:52:06.313088,19.97,NaN,NaN,NaN
3,1485494,271149,157175,bicycle,Juice Press,Juice Bar,OMG! My Favorite Juice!,1.0,Cold Pressed Juices,NaT,...,40.751257,-74.005634,2014-10-30 10:54:11.531894,2014-10-30 11:04:17.759577,2014-10-30 11:16:37.895816,2014-10-30 11:32:38.090061,NaN,10.103795,12.335604,16.003237
7,1311619,59161,79847,bicycle,Insomnia Cookies,Bakery,Chocolate Chunk,2.0,Cookies and Brownies,NaT,...,40.734703,-73.998206,2014-10-06 23:20:09.312584,2014-10-06 23:17:35.666950,2014-10-07 00:03:24.389592,2014-10-07 00:09:16.559521,NaN,-2.560761,45.812044,5.869499
8,1487674,55375,181543,bicycle,Cafe Zaiya,NaN,NaN,NaN,NaN,NaT,...,40.719758,-73.985011,2014-10-30 16:49:48.259935,2014-10-30 16:55:32.446053,2014-10-30 17:01:22.214264,2014-10-30 17:10:44.731901,NaN,5.736435,5.829470,9.375294
9,1417206,153816,157415,car,Shake Shack,Burger,Shackburger,1.0,Burgers,NaT,...,40.743613,-73.977684,2014-10-21 21:18:37.909545,2014-10-21 21:20:14.434227,2014-10-21 21:47:03.624931,2014-10-21 21:59:26.081278,NaN,1.608745,26.819845,12.374272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5974,1379770,243775,138061,car,Postmates Liquor Store,Shop,Raventos I Blanc L'Hereu Reserva Brut Cava - 2011,1.0,Sparkling,NaT,...,40.723861,-73.996981,2014-10-17 13:08:33.103688,2014-10-17 13:22:28.449114,2014-10-17 13:26:53.968347,2014-10-17 13:57:58.413231,NaN,13.922424,4.425321,31.074081
5976,1300266,135451,36664,bicycle,Juice Generation,Juice Bar,Red Dragon Fruit,1.0,Smoothies,NaT,...,40.778493,-73.986542,2014-10-05 13:48:36.871896,2014-10-05 13:54:59.502837,2014-10-05 14:06:07.389917,2014-10-05 14:13:57.894695,NaN,6.377182,11.131451,7.841746
5978,1360750,378035,151467,bicycle,Five Guys Burgers and Fries,Burger,Cheeseburger,1.0,Burgers,NaT,...,40.818637,-73.939241,2014-10-14 13:30:25.567577,2014-10-14 13:41:09.428090,2014-10-14 13:53:28.264588,2014-10-14 14:10:19.256853,NaN,10.731009,12.313942,16.849871
5979,1348697,96943,3296,bicycle,Cafe Mogador,Middle Eastern,Vegetarian,1.0,Cous Cous,NaT,...,40.725938,-73.980550,2014-10-12 18:37:42.565897,2014-10-12 18:44:21.331886,2014-10-12 18:54:16.442017,2014-10-12 19:00:43.047785,NaN,6.646100,9.918502,6.443429


In [33]:
data.how_long_it_took_to_order.describe()

count                           3038
unique                          2579
top       1900-01-01 00:03:20.155860
freq                               5
first     1900-01-01 00:01:22.997519
last      1900-01-01 01:13:13.266118
Name: how_long_it_took_to_order, dtype: object

In [34]:
print(str(round(100 - 2579/3038*100,2)) +'% of the data rows with the exact same time to deliver order(including miliseconds)')

15.11% of the data rows with the exact same time to deliver order(including miliseconds)


When examining the ‘how long it took to order’ column, only 2579 of data is unique. Means there are over 449 rows where ‘how long it took to order’ are duplicated elsewhere, same to precisely 4 decimals of milliseconds. 15.11% of this data is comprimised.

#### Checking the integrity of other columns to detect strange activity

In [35]:
data[data.how_long_it_took_to_order == '1900-01-01 00:03:20.155860']

,delivery_id,customer_id,jumpman_id,vehicle_type,pickup_place,place_category,item_name,item_quantity,item_category_name,how_long_it_took_to_order,...,dropoff_lat,dropoff_lon,when_the_delivery_started,when_the_Jumpman_arrived_at_pickup,when_the_Jumpman_left_pickup,when_the_Jumpman_arrived_at_dropoff,min_to_order,t_start_pickup_m,t_pickup_duration_m,t_dropoff_duration_m
2599,1460296,155145,39202,bicycle,La Esquina Taqueria,NaN,Bistec,1.0,Tacos,1900-01-01 00:03:20.155860,...,40.736889,-73.983087,2014-10-26 16:55:46.106588,2014-10-26 17:06:24.099110,2014-10-26 17:15:44.699287,2014-10-26 17:26:44.885842,3.33,10.633209,9.343336,11.003109
3510,1460296,155145,39202,bicycle,La Esquina Taqueria,NaN,Tortilla,1.0,Soups,1900-01-01 00:03:20.155860,...,40.736889,-73.983087,2014-10-26 16:55:46.106588,2014-10-26 17:06:24.099110,2014-10-26 17:15:44.699287,2014-10-26 17:26:44.885842,3.33,10.633209,9.343336,11.003109
3796,1460296,155145,39202,bicycle,La Esquina Taqueria,NaN,Chips & La Esquina's Own Salsa Roja,1.0,Plates,1900-01-01 00:03:20.155860,...,40.736889,-73.983087,2014-10-26 16:55:46.106588,2014-10-26 17:06:24.099110,2014-10-26 17:15:44.699287,2014-10-26 17:26:44.885842,3.33,10.633209,9.343336,11.003109
4056,1460296,155145,39202,bicycle,La Esquina Taqueria,NaN,Canned Soda,1.0,Cold Drinks,1900-01-01 00:03:20.155860,...,40.736889,-73.983087,2014-10-26 16:55:46.106588,2014-10-26 17:06:24.099110,2014-10-26 17:15:44.699287,2014-10-26 17:26:44.885842,3.33,10.633209,9.343336,11.003109
4487,1460296,155145,39202,bicycle,La Esquina Taqueria,NaN,Elotes Callejeros,1.0,Sides,1900-01-01 00:03:20.155860,...,40.736889,-73.983087,2014-10-26 16:55:46.106588,2014-10-26 17:06:24.099110,2014-10-26 17:15:44.699287,2014-10-26 17:26:44.885842,3.33,10.633209,9.343336,11.003109


It appears the only columns that have maintained integrity are the ‘item category name’ and ‘item_name’ columns. The rest of the data appears to scribe the events incorrectly. Because of the uniqueness of the ‘item_name’ column, it leads me to believe these other orders took place, but the events were hardcoded to save the attributes of previous events.

#### Examining Duration Interval Data

In [36]:
data[['t_start_pickup_m','t_pickup_duration_m','t_dropoff_duration_m']].describe()

,t_start_pickup_m,t_pickup_duration_m,t_dropoff_duration_m
count,5433.000000,5433.000000,5433.000000
mean,12.709215,18.916091,14.135259
std,12.289223,12.447806,9.301322
min,-2.651465,0.001877,0.839419
25%,3.595134,10.816819,7.998211
50%,9.985133,15.810259,11.770285
75%,18.966239,23.838006,17.396004
max,142.090125,267.954044,119.190060


Compromise in interval data, as it seems some of the data in jumpman_arrive_pickup is timestamped before the delivery starts(hence the -1 days in t_start_pickup), which is a problematic data integrity issue and has something to do with how the timestamp events are triggered.

### Conclusion

In terms of the data integrity issues, there are two things that should be done.

1.) Diagnose and fix the data scribing in the database to make sure the right data is being gathered.

2.) Eliminate the compromised rows of data for any further analysis.

As for the analysis already done, it should not have skewed the results significantly. It may have slightly skewed the proportions to which something may have occurred (exaggeration or minimization of amount of restaurants, preferred transport, concentrated heatzones in duplicated data).